In [1]:
import torch
from wavenet_model import *
from audio_data import WavenetDataset
from wavenet_training import *
from model_logging import *

# modified wavenet file implementation from: https://github.com/Vichoko/pytorch-wavenet/tree/master

dtype = torch.FloatTensor  # data type
ltype = torch.LongTensor  # label type

use_cuda = torch.cuda.is_available()
if use_cuda:
    print('use gpu')
    dtype = torch.cuda.FloatTensor
    ltype = torch.cuda.LongTensor

model = WaveNetModel(layers=10,
                     blocks=3,
                     dilation_channels=32,
                     residual_channels=32,
                     skip_channels=1024,
                     end_channels=512,
                     output_length=16,
                     dtype=dtype,
                     bias=True)
model.cuda()
print('model: ', model)
print('receptive field: ', model.receptive_field)
print('parameter count: ', model.parameter_count())

data = WavenetDataset(dataset_file='./example.npz',
                      item_length=model.receptive_field + model.output_length - 1,
                      target_length=model.output_length,
                      file_location='./unpacked_data',
                      test_stride=500)
print('the dataset has ' + str(len(data)) + ' items')


def generate_and_log_samples(step):
    sample_length = 32000
    gen_model = load_latest_model_from('snapshots', use_cuda=False)
    print("start generating...")
    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[0.5])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    # logger.audio_summary('temperature_0.5', tf_samples, step, sr=16000)

    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[1.])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    # logger.audio_summary('temperature_1.0', tf_samples, step, sr=16000)
    print("audio clips generated")


trainer = WavenetTrainer(model=model,
                         dataset=data,
                         lr=0.001,
                         snapshot_path='snapshots',
                         snapshot_name='birdset_model',
                         snapshot_interval=1000,
                         dtype=dtype,
                         ltype=ltype)

print('start training...')
start_data = data[250000][0] 
display(data)
trainer.train(batch_size=16,
              epochs=12)

use gpu
model:  WaveNetModel(
  (filter_convs): ModuleList(
    (0-29): 30 x Conv1d(32, 32, kernel_size=(2,), stride=(1,))
  )
  (gate_convs): ModuleList(
    (0-29): 30 x Conv1d(32, 32, kernel_size=(2,), stride=(1,))
  )
  (residual_convs): ModuleList(
    (0-29): 30 x Conv1d(32, 32, kernel_size=(1,), stride=(1,))
  )
  (skip_convs): ModuleList(
    (0-29): 30 x Conv1d(32, 1024, kernel_size=(1,), stride=(1,))
  )
  (start_conv): Conv1d(256, 32, kernel_size=(1,), stride=(1,))
  (end_conv_1): Conv1d(1024, 512, kernel_size=(1,), stride=(1,))
  (end_conv_2): Conv1d(512, 256, kernel_size=(1,), stride=(1,))
)
receptive field:  3070
parameter count:  1834592
one hot input
the dataset has 255490 items


C:\pj\dll24\notebooks\reproduce_training_ekaterina\gen_model_test\wavenet_model\wavenet_modules.py:53: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\tensor\python_tensor.cpp:80.)
  self.data = Variable(dtype(num_channels, max_length).zero_())


start training...
epoch 0
loss at step 50: 3.7523823404312133
one training step does take approximately 0.25975210666656495 seconds)
loss at step 100: 3.3060987424850463
loss at step 150: 3.11121160030365
loss at step 200: 3.0084674072265627


100%|██████████| 32/32 [00:03<00:00, 10.61it/s]


validation loss: 2.7252301275730133
validation accuracy: 20.751953125%
loss at step 250: 3.0322255754470824
loss at step 300: 2.982674684524536
loss at step 350: 2.926995987892151
loss at step 400: 2.9541908359527587


...

100%|██████████| 32/32 [00:02<00:00, 10.89it/s]


validation loss: 1.8637284226715565
validation accuracy: 34.75341796875%
loss at step 2850: 2.2385149598121643
loss at step 2900: 2.2489780449867247
loss at step 2950: 2.2974090433120726
loss at step 3000: 2.24168616771698


100%|██████████| 32/32 [00:02<00:00, 10.70it/s]


validation loss: 1.8640907481312752
validation accuracy: 34.3994140625%
loss at step 3050: 2.172415881156921
loss at step 3100: 2.202288272380829


KeyboardInterrupt: 

In [7]:
import torch
from wavenet_model import *
from audio_data import WavenetDataset
from wavenet_training import *
from model_logging import *
dtype = torch.FloatTensor  # data type
ltype = torch.LongTensor  # label type

use_cuda = torch.cuda.is_available()
if use_cuda:
    print('use gpu')
    dtype = torch.cuda.FloatTensor
    ltype = torch.cuda.LongTensor
device = torch.device("cuda")

model = WaveNetModel(layers=10,
                     blocks=3,
                     dilation_channels=32,
                     residual_channels=32,
                     skip_channels=1024,
                     end_channels=512,
                     output_length=16,
                     dtype=dtype,
                     bias=True)
model.load_state_dict(torch.load('birdset_modelwavenet_model.pth', weights_only=False))
model.eval()
model.cuda()

data = WavenetDataset(dataset_file='./example.npz',
                      item_length=model.receptive_field + model.output_length - 1,
                      target_length=model.output_length,
                      file_location='./unpacked_data',
                      test_stride=500)
print('the dataset has ' + str(len(data)) + ' items')



start_data = data[250000][0]
start_data = torch.max(start_data, 0)[1].to('cuda')
def prog_callback(step, total_steps):
    print(str(100 * step // total_steps) + "% generated")

generated = model.generate_fast(num_samples=32000,
                                 first_samples=start_data,
                                 progress_callback=prog_callback,
                                 progress_interval=1000,
                                 temperature=1.0,
                                 regularize=0.)

use gpu
one hot input
the dataset has 255490 items
torch.Size([1, 256, 1])
0% generated
2% generated
5% generated
8% generated
one generating step does take approximately 0.011234934329986573 seconds)
11% generated
14% generated
17% generated
19% generated
22% generated
25% generated
28% generated
31% generated
34% generated
37% generated
39% generated
42% generated
45% generated
48% generated
51% generated
54% generated
57% generated
59% generated
62% generated
65% generated
68% generated
71% generated
74% generated
76% generated
79% generated
82% generated
85% generated
88% generated
91% generated
94% generated
96% generated
99% generated


In [8]:
import IPython.display as ipd
import soundfile as sf

print(generated.shape)
ipd.Audio(generated, rate=16000)

sf.write('output_file.wav', generated, 16000)

(32000,)
